In [1]:
import torch
from torch.nn.functional import softmax
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from data.dataset import get_dataloaders
from train.deepspeed_sequence import TextClassificationTransformer

import numpy as np

/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
model_path = '../data/logs/e5-large/e5-epoch=00-val/F1Score=0.63.ckpt/checkpoint/mp_rank_00_model_states.pt'

In [3]:
num_classes = 2

model_name = "intfloat/multilingual-e5-large"

data_path = "../data/"

class_weight = torch.tensor([1.0, 4.6]) #torch.tensor([1.0, 2*4.60967657991111])
learning_rate = 5e-6
batch_size = 64

optimizer = opti_params = None
callbacks = []

num_classes = 2

criterion = torch.nn.CrossEntropyLoss(weight=class_weight)

In [4]:
model = TextClassificationTransformer(model_name, num_labels=num_classes, ignore_mismatched_sizes=True, cache_dir="../data/huggingface", loss=criterion)

/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
trainer = pl.Trainer(accelerator="gpu", devices=1, precision='16-mixed', logger=False)

/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /home/rasteiger/miniconda3/envs/dslab/lib/python3 ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
_, eval_loader = get_dataloaders(
    model_name=model_name, data_path=data_path, batch_size=batch_size, 
    use_cache=False, nickname='e5', comment_column='kommentar_original',
)

Map:   0%|          | 0/375708 [00:00<?, ? examples/s]

Map:   0%|          | 0/41746 [00:00<?, ? examples/s]

In [7]:
model.load_state_dict(torch.load(model_path)['module'])

<All keys matched successfully>

In [8]:
outputs = trainer.predict(model, dataloaders=eval_loader)

/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.8 /home/rasteiger/miniconda3/envs/dslab/lib/python3 ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environme

Predicting: |                                                                                                 …

TOKENIZERS_PARALLELISM=(true | false)


In [9]:
label_list=[]
for data in eval_loader:
    label_list.append(data['labels'].numpy())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [13]:
probabilities = np.concatenate([softmax(x[0]['logits'].float(), dim=-1).numpy() for x in outputs])
predictions = np.argmax(probabilities, axis=-1)
labels = np.concatenate([x[1].numpy() for x in outputs])

In [14]:
from sklearn.metrics import f1_score

In [17]:
f1_score(predictions, labels, average='macro'), f1_score(predictions, labels, average='binary')

(0.7578002303937157, 0.6298252742787486)